# Random Forest Regression

In [1]:
from train_utils import *
import torch
from torch.utils.data import DataLoader
from dataset.dataset_loader import SNDataset, SNDatasetClimate, myNormalize, myToTensor, Augmentations, RFTransform, TensorCenterPixels
from torchvision import transforms
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import date, datetime
import json


In [2]:
# create a folder called 'results' in the current directory if it doesn't exist
if not os.path.exists('results'):
    os.mkdir('results')

In [3]:
# Format the date and time
now = datetime.now()
start_string = now.strftime("%Y-%m-%d %H:%M:%S")
file_name = now.strftime("D_%Y_%m_%d_T_%H_%M")
print("File Name:", file_name)
print("Current Date and Time:", start_string)

File Name: D_2024_02_27_T_09_30
Current Date and Time: 2024-02-27 09:30:06


In [4]:
import os
os.getcwd()

NAFISEH = "Nafiseh"
MOIEN = "Moien"

if "d:" in os.getcwd():
    USER = MOIEN
elif "c:" in os.getcwd():
    USER = NAFISEH
else:
    raise Exception("Unknown user")

USER

'Nafiseh'

In [5]:
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
OC_MAX = 87
USE_CLIMATE = True
USE_SRTM = True

### No Normalization
Random Forest doens't need normalization. So I addedthe RF transfom, it only reshapes the image into channels first format.
then used myTransfomr to resize to 64x64.

You can test my Normalize transform by uncommenting the line in the cell below.

### Cut Center
Cuts a 2x2 square from the center of the image.
If `interplate_center_pixel` is set to True, then the center pixel is interpolated from the 4 surrounding pixels.

In [7]:
#mynorm = myNormalize(img_bands_min_max =[[(0,7),(0,1)], [(7,12),(-1,1)], [(12), (-4,2963)], [(13), (0, 90)]], oc_min = 0, oc_max = 200)
rf_transform = RFTransform(oc_min = 0, oc_max = OC_MAX)
my_to_tensor = myToTensor()
cut_center = TensorCenterPixels(pixel_radius=1 ,interpolate_center_pixel = False)
transform = transforms.Compose([rf_transform,my_to_tensor,cut_center])

### Bands to use

In [8]:
bands = [0,1,2,3,4,5,6,7,8,9,10,11] if not USE_SRTM else [0,1,2,3,4,5,6,7,8,9,10,11,12,13]

########################################################################################
################################# IF Not USE_CLIMATE ###############################
########################################################################################

if not USE_CLIMATE: # NOT USING THE CLIMATE DATA
    if USER == MOIEN:
        train_ds = SNDataset('D:\python\SoilNet\dataset\l8_images\\train\\','D:\python\SoilNet\dataset\LUCAS_2015_all.csv',l8_bands=bands, transform=transform)
    elif USER == NAFISEH:
        train_ds = SNDataset('C:\\Users\\nkakhani\\_Multimodal\\SoilNet-3\\SoilNet\\dataset\\l8_images\\train',\
                             'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-3\\SoilNet\\dataset\\LUCAS_2015_all.csv',l8_bands=bands, transform=transform) #Nafiseh 
    if USER == MOIEN:
        test_ds = SNDataset('D:\python\SoilNet\dataset\l8_images\\test\\','D:\python\SoilNet\dataset\LUCAS_2015_all.csv',
                            l8_bands=bands, transform=transform, return_point_id=True)
    elif USER == NAFISEH:
        test_ds = SNDataset('C:\\Users\\nkakhani\\_Multimodal\\SoilNet-3\\SoilNet\\dataset\\l8_images\\test',\
                            'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-3\\SoilNet\\dataset\\LUCAS_2015_all.csv',
                            l8_bands=bands, transform=transform,return_point_id=True) #Nafiseh 
        
########################################################################################
################################### IF USE_CLIMATE #################################
########################################################################################
else: # USING THE CLIMATE DATA
    if USER == MOIEN:
        train_ds = SNDatasetClimate('D:\python\SoilNet\dataset\l8_images\\train\\',
                                    'D:\python\SoilNet\dataset\LUCAS_2015_all.csv',
                                    "D:\\python\\SoilNet\\dataset\\Climate\\All\\filled\\",
                                    l8_bands=bands, transform=transform, normalize_climate = False)
    elif USER == NAFISEH:
        train_ds = SNDatasetClimate('C:\\Users\\nkakhani\\_Multimodal\\SoilNet-3\\SoilNet\\dataset\\l8_images\\train',\
                            'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-3\\SoilNet\\dataset\\LUCAS_2015_all.csv',
                            'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-3\\SoilNet\\dataset\\Climate\\All\\filled',
                            l8_bands=bands, transform=transform, normalize_climate = False) #Nafiseh 
    if USER == MOIEN:
        test_ds = SNDatasetClimate('D:\python\SoilNet\dataset\l8_images\\test\\',
                                'D:\python\SoilNet\dataset\LUCAS_2015_all.csv',
                                "D:\\python\\SoilNet\\dataset\\Climate\\All\\filled\\",
                                l8_bands=bands, transform=transform, normalize_climate = False, return_point_id=True)
    elif USER == NAFISEH:
        test_ds = SNDatasetClimate('C:\\Users\\nkakhani\\_Multimodal\\SoilNet-3\\SoilNet\\dataset\\l8_images\\test',\
                            'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-3\\SoilNet\\dataset\\LUCAS_2015_all.csv',
                            'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-3\\SoilNet\\dataset\\Climate\\All\\filled',
                            l8_bands=bands, transform=transform, normalize_climate = False, return_point_id=True) #Nafiseh

In [9]:
train_ds[0][0][0].shape, train_ds[0][0][1].shape

(torch.Size([14, 2, 2]), torch.Size([61, 11]))

In [10]:
# CONFIG
NUM_WORKERS = 6 if USER == NAFISEH else 2
TRAIN_BATCH_SIZE = 32 if USER == NAFISEH else 4
TEST_BATCH_SIZE = 32 if USER == NAFISEH else 4


In [11]:
train_dl = DataLoader(train_ds, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_dl = DataLoader(test_ds, batch_size=TEST_BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

In [12]:
if USE_CLIMATE:
    # Preprocess the data using the DataLoader
    X_processed = []
    y_processed = []
    for batch_idx, (features, target) in enumerate(train_dl):
        images_np = features[0].numpy()
        climate_np = features[1].numpy()
        # Preprocess the features as needed
        images_processed = images_np.reshape(images_np.shape[0], -1) # Flatten the images with shape (batch_size, num_pixels * num_bands) -> e.g: (32, 4 * 12) if 4 pixel is being used or (32, 1 * 12) if 1 pixel is being used 
        climate_processed = climate_np.reshape(climate_np.shape[0], -1) # Flatten the climate data with shape (batch_size, num_climate_features * sequence_length) -> e.g: (32, 14*61) if 14 climate feature is being used and the each feature is a sequence of 61 months
        features_processed = np.concatenate([images_processed, climate_processed], axis=1)
        X_processed.append(features_processed)
        y_processed.append(target.numpy())

    X_processed = np.concatenate(X_processed, axis=0)  # (DataLoader Length, num_pixels * num_bands + num_climate_features * sequence_length)
    y_processed = np.concatenate(y_processed, axis=0)  # (DataLoader Length,)
else:
    # Preprocess the data using the DataLoader
    X_processed = []
    y_processed = []
    for batch_idx, (features, target) in enumerate(train_dl):
        features_np = features.numpy()
        # Preprocess the features as needed
        features_processed = features_np.reshape(features_np.shape[0], -1)
        X_processed.append(features_processed)
        y_processed.append(target.numpy())

    X_processed = np.concatenate(X_processed, axis=0)
    y_processed = np.concatenate(y_processed, axis=0)

In [13]:
print(X_processed.shape, X_processed.dtype,"|",y_processed.shape, y_processed.dtype)
print(f"Memory size of the Train array is {X_processed.nbytes/(1024**2)} MB or {X_processed.nbytes/(1024**3)} GB" )

(15296, 727) float32 | (15296,) float32
Memory size of the Train array is 42.420166015625 MB or 0.04142594337463379 GB


## Grid Search.
I don't know what are the best parameters for the random forest. <span style="color: green;">Please change them and let me know what works best</span>. Thank you

In [14]:
# Define the grid of hyperparameters to search over
# param_dist = {
#     # 'n_estimators': randint(30, 1000),
#     'n_estimators': [1, 2, 5, 10, 20, 30, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500],
#     'max_depth': [1, 2, 3, 4, 5, 10],
#     # 'max_features': [1.0, 'sqrt'],
#     'min_samples_split': [1, 2, 5, 10],
#     'min_samples_leaf': [1, 2, 5, 10, 15, 20],
#     'max_leaf_nodes': [2, 5, 10, 15, 20],
# }

### Larger Grid

In [15]:
# # DEEP SEARCH
# param_grid = {
#     'n_estimators': [10, 20, 30],
#     'max_depth': [None, 5, 10, 20, 30],
#     'max_features': ['sqrt', 'log2'],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'criterion': ['mse', 'mae'],
#     'bootstrap': [True, False],
#     'oob_score': [True, False],
#     'max_samples': [0.5, 0.75, None],
#     'max_leaf_nodes': [None, 10, 20],
#     'min_impurity_decrease': [0.0, 0.1],
#     'ccp_alpha': [0.0, 0.1],
#     'warm_start': [True, False]
# }

In [16]:
NUM_ITTERS = 1 if USER == NAFISEH else 1
CV = 2 if USER == NAFISEH else 2
SEED = 42

In [17]:
# # Define RandomForestRegressor
# rfr_ = RandomForestRegressor()
# # Define the randomized search object
# rfr = RandomizedSearchCV(
#     estimator=rfr_,
#     param_distributions=param_dist,
#     n_iter=NUM_ITTERS, # Number of Combinations from the grid to try
#     cv=CV, # Corss Validation Folds
#     random_state=SEED
# )

In [18]:
from sklearn.model_selection import cross_val_predict

# Define RandomForestRegressor with the desired parameters
rfr = RandomForestRegressor(
    n_estimators = 300,
    max_depth = 10,
    min_samples_split = 5,
    min_samples_leaf = 20,
    max_leaf_nodes = 20,
)

In [19]:
rfr.fit(X_processed, y_processed)

RandomForestRegressor(max_depth=10, max_leaf_nodes=20, min_samples_leaf=20,
                      min_samples_split=5, n_estimators=300)

In [20]:
# print(rfr.best_params_)

### Testing a Random Image

In [21]:
if USE_CLIMATE:
    # Use the trained model to predict on a new image
    n_rand = np.random.randint(0, len(test_ds))
    new_image = test_ds[n_rand][0][0].numpy()
    new_climate = test_ds[n_rand][0][1].numpy()
    point_id = test_ds[n_rand][2]
    print("Point ID: ", point_id, "type: ", type(point_id))
    new_image_processed = new_image.reshape(1, -1)
    new_climate_processed = new_climate.reshape(1, -1)
    new_features_processed = np.concatenate([new_image_processed, new_climate_processed], axis=1)
    y_pred = rfr.predict(new_features_processed)
    print("y_pred: ", y_pred[0], "|" ,"y_true: ", test_ds[n_rand][1].numpy())
else:
    # Use the trained model to predict on a new image
    n_rand = np.random.randint(0, len(test_ds))
    new_image = test_ds[n_rand][0].numpy()
    new_image_processed = new_image.reshape(1, -1)
    y_pred = rfr.predict(new_image_processed)
    point_id = test_ds[n_rand][2]
    print("Point ID: ", point_id, "type: ", type(point_id))
    print("y_pred: ", y_pred[0], "|" ,"y_true: ", test_ds[n_rand][1].numpy())

Point ID:  52944122 type:  <class 'str'>
y_pred:  48.56009949370841 | y_true:  32.1


## RMSE for the whole dataset.

In [22]:
if USE_CLIMATE:
    # Preprocess the data using the DataLoader
    X_processed = []
    y_processed = []
    point_id_list = []
    for batch_idx, (features, target,point_id) in enumerate(test_dl):
        images_np = features[0].numpy()
        climate_np = features[1].numpy()
        
        # Preprocess the features as needed
        images_processed = images_np.reshape(images_np.shape[0], -1) # Flatten the images with shape (batch_size, num_pixels * num_bands) -> e.g: (32, 4 * 12) if 4 pixel is being used or (32, 1 * 12) if 1 pixel is being used 
        climate_processed = climate_np.reshape(climate_np.shape[0], -1) # Flatten the climate data with shape (batch_size, num_climate_features * sequence_length) -> e.g: (32, 14*61) if 14 climate feature is being used and the each feature is a sequence of 61 months
        features_processed = np.concatenate([images_processed, climate_processed], axis=1)
        X_processed.append(features_processed)
        y_processed.append(target.numpy())
        point_id_list = point_id_list + list(point_id)

    X_processed = np.concatenate(X_processed, axis=0)  # (DataLoader Length, num_pixels * num_bands + num_climate_features * sequence_length)
    y_processed = np.concatenate(y_processed, axis=0)  # (DataLoader Length,)
else:
    # Preprocess the data using the DataLoader
    X_processed = []
    y_processed = []
    point_id_list = []
    for batch_idx, (features, target,point_id) in enumerate(test_dl):
        features_np = features.numpy()
        # Preprocess the features as needed
        features_processed = features_np.reshape(features_np.shape[0], -1)
        X_processed.append(features_processed)
        y_processed.append(target.numpy())
        point_id_list = point_id_list + list(point_id)

    X_processed = np.concatenate(X_processed, axis=0)
    y_processed = np.concatenate(y_processed, axis=0)

In [23]:
point_id_list = [int(i) for i in point_id_list]
point_id_arr = np.array(point_id_list)

In [24]:
y_pred = rfr.predict(X_processed)

In [25]:
print(len(point_id_arr), len(y_processed), len(y_pred))

4371 4371 4371


In [26]:
# RSME
rmse = np.sqrt(mean_squared_error(y_processed, y_pred))
print('RMSE:', rmse)
# MAE
mae = mean_absolute_error(y_processed, y_pred)
print('MAE:', mae)

RMSE: 21.685791523723978
MAE: 16.335132646669223


In [27]:
# Save y_pred and y_true, point_id to a csv file
df = pd.DataFrame({'point_id': point_id_list, 'y_true': y_processed, 'y_pred': y_pred})
df.to_csv(f"results/RF_{file_name}_{USER}.csv", index=False)

In [28]:
# Format the date and time
now = datetime.now()
finish_string = now.strftime("%Y-%m-%d %H:%M:%S")
print("Finish Date and Time:", finish_string)

Finish Date and Time: 2023-06-06 13:38:20


In [29]:
log_json = {}
log_json['RMSE'] = rmse
log_json['MAE'] = mae
log_json['USE_CLIMATE'] = USE_CLIMATE
log_json['USE_SRTM'] = USE_SRTM

log_json['NUM_ITTERS'] = NUM_ITTERS
log_json['CV'] = CV
log_json['SEED'] = SEED

log_json['param_dist'] = str(param_dist)
log_json['BEST_PARAMS'] = rfr.best_params_

log_json['TIME'] = {'start': start_string, 'finish': finish_string}

log_json

NameError: name 'param_dist' is not defined

In [ ]:
with open(f"results/RF_{file_name}_{USER}.json", "w") as fp:
    json.dump(log_json, fp, indent=4)